In [1]:
from sympy import Matrix, Symbol, derive_by_array, Lambda, Function, MatrixSymbol, Derivative
from sympy import var
from sympy.abc import x, i, j, a, b, c, d




In [2]:
def myvar(letter: str, i: int, j: int) -> Symbol:
    letter_ij = Symbol('{}_{}{}'.format(letter, i+1, j+1), is_commutative=True)
    return letter_ij


ns,ms,ps = 3,3,2

X = Matrix(ns, ms, lambda i,j : myvar('x', i, j)); X

Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]])

In [3]:
W = Matrix(ms, ps, lambda i,j : myvar('w', i, j)); W

Matrix([
[w_11, w_12],
[w_21, w_22],
[w_31, w_32]])

In [4]:
#TODO how to make matrix symbols commutative?
# A = MatrixSymbol('X',ns,ms, is_commutative=True); Matrix(A)
A = MatrixSymbol('X',ns,ms); Matrix(A)
B = MatrixSymbol('W',ms,ps)







In [5]:
v = lambda a,b: a*b

vL = Lambda((a,b), a*b)

n = Function('v') #, Lambda((a,b), a*b))

vN = lambda mat1, mat2: Matrix(mat1.shape[0], mat2.shape[1], lambda i, j: Symbol("n_{}{}".format(i+1, j+1))); vN


Nelem = vN(X, W)
Nelem

Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])

In [6]:
Nspec = v(X,W)
Nspec

Matrix([
[w_11*x_11 + w_21*x_12 + w_31*x_13, w_12*x_11 + w_22*x_12 + w_32*x_13],
[w_11*x_21 + w_21*x_22 + w_31*x_23, w_12*x_21 + w_22*x_22 + w_32*x_23],
[w_11*x_31 + w_21*x_32 + w_31*x_33, w_12*x_31 + w_22*x_32 + w_32*x_33]])

In [7]:
#N = v(X, W); N
N = n(A,B)
N









v(X, W)

In [8]:

def siga(mat: Matrix) -> Matrix:
     #lst = mat.tolist()
     nr, nc = mat.shape

     applied = [[sigma(mat[i,j]) for j in range(0, nc)] for i in range(0, nr)]

     return Matrix(applied)


# way 2 of declaring S (better way)
sigma = Function('sigma')

sigmaApply = Function("sigma_apply") #lambda matrix:  matrix.applyfunc(sigma)

sigmaApply_ = lambda matrix: matrix.applyfunc(sigma)

sigmaApply_2 = lambda matrix: siga(matrix)

S = sigmaApply(N); S

sigma_apply(v(X, W))

In [9]:
sigmaApply_(Nelem)

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [10]:
sigmaApply_2(Nelem)

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [11]:
#sigmaApply_2(A*B).diff(Matrix(A))

In [12]:
Sspec = S.subs({A:X, B:W}).replace(n, v).replace(sigmaApply, sigmaApply_)
Sspec

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [13]:
Selem = S.replace(n, vN).replace(sigmaApply, sigmaApply_)
Selem

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [14]:
import itertools

elemToSpecD = dict(itertools.chain(*[[(Nelem[i, j], Nspec[i, j]) for j in range(2)] for i in range(3)]))

elemToSpec = list(elemToSpecD.items())

Matrix(elemToSpec)

Matrix([
[n_11, w_11*x_11 + w_21*x_12 + w_31*x_13],
[n_12, w_12*x_11 + w_22*x_12 + w_32*x_13],
[n_21, w_11*x_21 + w_21*x_22 + w_31*x_23],
[n_22, w_12*x_21 + w_22*x_22 + w_32*x_23],
[n_31, w_11*x_31 + w_21*x_32 + w_31*x_33],
[n_32, w_12*x_31 + w_22*x_32 + w_32*x_33]])

In [15]:
elemToSpecFuncD = dict(itertools.chain(*[[(Nelem[i, j], Function("n_{}{}".format(i + 1, j + 1))(Nspec[i, j])) for j in range(2)] for i in range(3)]))

elemToSpecFunc = list(elemToSpecFuncD.items())

Matrix(elemToSpecFunc)

Matrix([
[n_11, n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)],
[n_12, n_12(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[n_21, n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)],
[n_22, n_22(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[n_31, n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)],
[n_32, n_32(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [16]:
elemToSpecFuncArgsD = dict(itertools.chain(*[[(Nelem[i, j], Function("n_{}{}".format(i + 1, j + 1))(*X,*W)) for j in range(2)] for i in range(3)]))

elemToSpecFuncArgs = list(elemToSpecFuncArgsD.items())

Matrix(elemToSpecFuncArgs)

Matrix([
[n_11, n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_12, n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_21, n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_22, n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_31, n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_32, n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)]])

In [17]:
elemToMatArgD = dict(itertools.chain(*[[(Nelem[i, j], Function("n_{}{}".format(i+1,j+1))(A,B) ) for j in range(2)] for i in range(3)]))

elemToMatArg = list(elemToMatArgD.items())

Matrix(elemToMatArg)

Matrix([
[n_11, n_11(X, W)],
[n_12, n_12(X, W)],
[n_21, n_21(X, W)],
[n_22, n_22(X, W)],
[n_31, n_31(X, W)],
[n_32, n_32(X, W)]])

In [18]:
matargToSpecD = dict(zip(elemToMatArgD.values(), elemToSpecD.values()))

matargToSpec = list(matargToSpecD.items())

Matrix(matargToSpec)

Matrix([
[n_11(X, W), w_11*x_11 + w_21*x_12 + w_31*x_13],
[n_12(X, W), w_12*x_11 + w_22*x_12 + w_32*x_13],
[n_21(X, W), w_11*x_21 + w_21*x_22 + w_31*x_23],
[n_22(X, W), w_12*x_21 + w_22*x_22 + w_32*x_23],
[n_31(X, W), w_11*x_31 + w_21*x_32 + w_31*x_33],
[n_32(X, W), w_12*x_31 + w_22*x_32 + w_32*x_33]])

In [19]:
Selem

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [20]:
Selem.subs(elemToSpecD)

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [21]:
Selem[0,1].diff(Nelem[0,1])

Derivative(sigma(n_12), n_12)

In [22]:
Selem[0,1].diff(Nelem[0,1]).subs({Nelem[0,1] : Nspec[0,1]})
#Selem[0,1].diff(Nelem[0,1]).subs(dict([{Nelem[0,1] : Nspec[0,1]}]))

Subs(Derivative(sigma(n_12), n_12), n_12, w_12*x_11 + w_22*x_12 + w_32*x_13)

In [23]:
Selem[0,1].diff(Nelem[0,1]).subs({Nelem[0,1] : Nspec[0,1]}).subs({Nspec[0,1] : 23})

Subs(Derivative(sigma(n_12), n_12), n_12, 23)

In [24]:
Selem[0,1].diff(Nelem[0,1]).subs({Nelem[0,1] : Nspec[0,1]}).replace(sigma, lambda x: 8*x**3)

Subs(Derivative(8*n_12**3, n_12), n_12, w_12*x_11 + w_22*x_12 + w_32*x_13)

In [25]:
Selem[0,1].diff(Nelem[0,1]).replace(sigma, lambda x: 8*x**3)

Derivative(8*n_12**3, n_12)

In [26]:
Selem[0,1].diff(Nelem[0,1]).replace(sigma, lambda x: 8*x**3).doit()

24*n_12**2

In [27]:
# ### GOT IT: can replace now with expression and do derivative with respect to that expression.
Selem[0,1].diff(Nelem[0,1]).subs({Nelem[0,1] : Nspec[0,1]}).replace(sigma, lambda x: 8*x**3).doit()

24*(w_12*x_11 + w_22*x_12 + w_32*x_13)**2

In [28]:
Selem[0,1].subs({Nelem[0,1] : Nspec[0,1]}).diff(X[0,1])#.subs({Nelem[0,1] : Nspec[0,1]})

w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_11 + w_22*x_12 + w_32*x_13)

In [29]:
Selem

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [30]:
nt = Nelem.subs(elemToSpecFunc); nt

Matrix([
[n_11(w_11*x_11 + w_21*x_12 + w_31*x_13), n_12(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[n_21(w_11*x_21 + w_21*x_22 + w_31*x_23), n_22(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[n_31(w_11*x_31 + w_21*x_32 + w_31*x_33), n_32(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [31]:
st = Selem.subs(elemToSpecFunc); st

Matrix([
[sigma(n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), sigma(n_12(w_12*x_11 + w_22*x_12 + w_32*x_13))],
[sigma(n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)), sigma(n_22(w_12*x_21 + w_22*x_22 + w_32*x_23))],
[sigma(n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)), sigma(n_32(w_12*x_31 + w_22*x_32 + w_32*x_33))]])

In [32]:
st.diff(nt)

[[[[Derivative(sigma(n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), 0], [0, 0], [0, 0]], [[0, Derivative(sigma(n_12(w_12*x_11 + w_22*x_12 + w_32*x_13)), n_12(w_12*x_11 + w_22*x_12 + w_32*x_13))], [0, 0], [0, 0]]], [[[0, 0], [Derivative(sigma(n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)), n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)), 0], [0, 0]], [[0, 0], [0, Derivative(sigma(n_22(w_12*x_21 + w_22*x_22 + w_32*x_23)), n_22(w_12*x_21 + w_22*x_22 + w_32*x_23))], [0, 0]]], [[[0, 0], [0, 0], [Derivative(sigma(n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)), n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)), 0]], [[0, 0], [0, 0], [0, Derivative(sigma(n_32(w_12*x_31 + w_22*x_32 + w_32*x_33)), n_32(w_12*x_31 + w_22*x_32 + w_32*x_33))]]]]

In [33]:
st[0,0].diff(st[0,0].args[0])

Derivative(sigma(n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), n_11(w_11*x_11 + w_21*x_12 + w_31*x_13))

In [34]:
temp = st[0,0].diff(X[0,0]); temp

#nt[0,0]

#temp.replace(Function("n_11")(nt[0,0].args[0]), nt[0,0].args[0])

#temp.subs({nt[0,0] : nt[0,0].args[0]})



w_11*Derivative(sigma(n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), n_11(w_11*x_11 + w_21*x_12 + w_31*x_13))*Subs(Derivative(n_11(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13)

In [35]:
st[0,0].diff(st[1,0].args[0])

0

In [36]:
Selem.diff(Nelem)

[[[[Derivative(sigma(n_11), n_11), 0], [0, 0], [0, 0]], [[0, Derivative(sigma(n_12), n_12)], [0, 0], [0, 0]]], [[[0, 0], [Derivative(sigma(n_21), n_21), 0], [0, 0]], [[0, 0], [0, Derivative(sigma(n_22), n_22)], [0, 0]]], [[[0, 0], [0, 0], [Derivative(sigma(n_31), n_31), 0]], [[0, 0], [0, 0], [0, Derivative(sigma(n_32), n_32)]]]]

In [37]:
Selem.diff(Nelem).subs(elemToSpecFunc)

[[[[Derivative(sigma(n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), 0], [0, 0], [0, 0]], [[0, Derivative(sigma(n_12(w_12*x_11 + w_22*x_12 + w_32*x_13)), n_12(w_12*x_11 + w_22*x_12 + w_32*x_13))], [0, 0], [0, 0]]], [[[0, 0], [Derivative(sigma(n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)), n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)), 0], [0, 0]], [[0, 0], [0, Derivative(sigma(n_22(w_12*x_21 + w_22*x_22 + w_32*x_23)), n_22(w_12*x_21 + w_22*x_22 + w_32*x_23))], [0, 0]]], [[[0, 0], [0, 0], [Derivative(sigma(n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)), n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)), 0]], [[0, 0], [0, 0], [0, Derivative(sigma(n_32(w_12*x_31 + w_22*x_32 + w_32*x_33)), n_32(w_12*x_31 + w_22*x_32 + w_32*x_33))]]]]

In [38]:
# CAN even replace elements after have done an operation on them!!! replacing n_21 * 2 with the number 4.
Sspec.subs({Nspec[0, 0]: 3}).replace(sigma, lambda x: 2 * x).replace(Nspec[2, 1] * 2, 4)









Matrix([
[                                      6, 2*w_12*x_11 + 2*w_22*x_12 + 2*w_32*x_13],
[2*w_11*x_21 + 2*w_21*x_22 + 2*w_31*x_23, 2*w_12*x_21 + 2*w_22*x_22 + 2*w_32*x_23],
[2*w_11*x_31 + 2*w_21*x_32 + 2*w_31*x_33,                                       4]])

In [39]:
lambd = Function("lambda")
lambd_ = lambda matrix : sum(matrix)

L = lambd(S); L

lambda(sigma_apply(v(X, W)))

In [40]:
L.replace(n, vN).replace(sigmaApply, sigmaApply_)

lambda(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))

In [41]:
#L.replace(n, vN).replace(sigmaApply, sigmaApply_).diff(Nelem[0,0])

In [42]:
Lsum = L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_)
Lsum

sigma(n_11) + sigma(n_12) + sigma(n_21) + sigma(n_22) + sigma(n_31) + sigma(n_32)

In [43]:
Lsum.diff(Nelem)

Matrix([
[Derivative(sigma(n_11), n_11), Derivative(sigma(n_12), n_12)],
[Derivative(sigma(n_21), n_21), Derivative(sigma(n_22), n_22)],
[Derivative(sigma(n_31), n_31), Derivative(sigma(n_32), n_32)]])

In [44]:
Lsum.subs(elemToSpecD)#.diff(X[2,1])

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13) + sigma(w_11*x_21 + w_21*x_22 + w_31*x_23) + sigma(w_11*x_31 + w_21*x_32 + w_31*x_33) + sigma(w_12*x_11 + w_22*x_12 + w_32*x_13) + sigma(w_12*x_21 + w_22*x_22 + w_32*x_23) + sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)

In [45]:
Lsum.subs(elemToSpecD).diff(X)

Matrix([
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_11 + w_22*x_12 + w_32*x_13), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_11 + w_22*x_12 + w_32*x_13), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13) + w_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_11 + w_22*x_12 + w_32*x_13)],
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_21 + w_21*x_22 + w_31*x_23) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_21 + w_22*x_22 + w_32*x_23), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_21 + w_21*x_22 + w_31*x_23) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_21 + w_22*x_22 + w_32*x_23), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_21 + w_21*x_22 + w_31*x_23) + w_32*Subs(Derivative(sigma(_xi_1), _

In [46]:
# METHOD 1: direct matrix diff
#
### END RESULT ACHIEVED HERE (this is the end result and the most specific form of the result of the  matrix differentiation, when sigma is unknown)
specToElemD = {v:k for k,v in elemToSpecD.items()}

assert Lsum == L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_)
Lsum.subs(elemToSpecD).diff(X).subs(specToElemD)



Matrix([
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + w_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12)],
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + w_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22)],
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31) + w_32*Sub

In [47]:
# METHOD 2: doing matrix symbol diff
#
#### NOW DOING THE MATRIX SYMBOL DIFF EXPRESSION (trying to achieve a form that shows the chain rule w.r.t to matrix symbol)
Selem

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [48]:
L

lambda(sigma_apply(v(X, W)))

In [49]:
#L.replace(A, A.T).diff(A) #ERROR: fatal python error ... why??

In [50]:
#L.replace(n,v).diff(A).replace(sigmaApply, sigmaApply_) # ERROR
#L.replace(n,vN).subs(elemToSpecFuncD).replace(sigmaApply, sigmaApply_).diff(X) # why the zero matrix?

In [51]:
L.replace(n,v).diff(A)

Subs(Derivative(sigma_apply(_xi_1), _xi_1), _xi_1, X*W)*Derivative(lambda(sigma_apply(X*W)), sigma_apply(X*W))*Derivative(X*W, X)

In [52]:
L.replace(n,vL).diff(A)

Subs(Derivative(sigma_apply(_xi_1), _xi_1), _xi_1, X*W)*Derivative(lambda(sigma_apply(X*W)), sigma_apply(X*W))*Derivative(X*W, X)

In [53]:
#L.replace(n,v).diff(A).replace(lambd,lambd_) ### ERROR sigma object is not iterable
#L.replace(n,vL).diff(A).replace(sigmaApply, sigmaApply_)### ERROR
#L.replace(n,v).diff(A).replace(sigmaApply, sigmaApply_) ### ERROR dummy object has no attribute applyfunc

In [54]:
#L.replace(sigmaApply, sigmaApply_).diff(A) # ERROR
# L.replace(lambd, lambd_) # ERROR

#L.replace(n, v).replace(sigmaApply, sigmaApply_2)# shows matrix results, too specific, want the function composition notation as below but just applied to the function v(X,W) in abstract way
### METHOD 0: (prepare by substituting n --> v, then sigmaApply --> sigma)
L.replace(n, v).replace(sigmaApply, sigmaApply_)#.replace(lambd, lambd_)

lambda(Lambda(_d, sigma(_d)).(X*W))

In [55]:

# NOTE: the point here is that even replacing with a sympy Lambda doesn't give same result as above since above uses the V.applyfunc(sigma) within the Lambda.
L.replace(sigmaApply, Lambda(d, sigma(d)))

lambda(sigma(v(X, W)))

In [56]:

vSym = Symbol('v', applyfunc=True)
L.replace(n(A,B), vSym)

lambda(sigma_apply(v))

In [57]:
#L.replace(n(A,B), vSym).replace(sigmaApply, sigmaApply_)# ERROR because Symbol has no atttribute applyfunc (that is the one we want though so must use matrix symbol which for some reason works instead of just an ordinary symbol v
#V = MatrixSymbol()
# Takes in the symbols A and B matrices and returns the matrix symbol with the shape that is supposed to result after A*B
V = lambda matA, matB: MatrixSymbol('V', matA.shape[0], matB.shape[1])
V
V(A,B)#.shape

V

In [58]:
from sympy import symbols
#V = MatrixSymbol('V', X.shape[0], W.shape[1])
i, j = symbols('i j')
M = MatrixSymbol('M', i, j)# abstract shape
sigmaApply_L = Lambda(M, M.applyfunc(sigma))
lambda_L = Lambda(M, sum(M))

In [59]:
sigmaApply_L(A)

Lambda(_d, sigma(_d)).(X)

In [60]:
# TODO: trying to figure out how to write L so that it is in terms of lambdas so get the form (d ---> sigma(d) COMPOAED ((X,W) -> V)) instead of (sigmaApply(v(X,W)))
Vs = MatrixSymbol("Vs", A.shape[0], B.shape[1])
VL = Lambda((A,B), MatrixSymbol('V', A.shape[0], B.shape[1]))
VL

Lambda((X, W), V)

In [61]:
L.replace(n, VL)#.replace(sigmaApply, sigmaApply_L).subs({V:VL})

lambda(sigma_apply(V))

In [62]:
L.replace(n, VL).replace(sigmaApply, sigmaApply_)#.subs({VL(A,B) : n(A,B)}) ### ERROR
# This is v(X,W) in Lambda form:
VL

Lambda((X, W), V)

In [63]:
VL(A,B)
#L.subs({n: V})

V

In [64]:
L.replace(n(A,B), VL(A,B))#.replace(sigmaApply, sigmaApply_).subs({V(A,B) : n})

lambda(sigma_apply(V))

In [65]:
lambd(sigmaApply(VL))

lambda(sigma_apply(Lambda((X, W), V)))

In [66]:
lambd(sigmaApply(VL)).replace(VL, n(A,B))

lambda(sigma_apply(v(X, W)))

In [67]:
lambd(sigmaApply(VL)).diff(A)

Subs(Derivative(sigma_apply(_xi_1), _xi_1), _xi_1, Lambda((X, W), V))*Derivative(lambda(sigma_apply(Lambda((X, W), V))), sigma_apply(Lambda((X, W), V)))*Derivative(Lambda((X, W), V), X)

In [68]:
lambd(sigmaApply(VL)).diff(A).replace(VL, n(A,B))

Subs(Derivative(sigma_apply(_xi_1), _xi_1), _xi_1, v(X, W))*Derivative(lambda(sigma_apply(v(X, W))), sigma_apply(v(X, W)))*Derivative(v(X, W), X)

In [69]:
lambd(sigmaApply(VL))#.replace(sigmaApply, sigmaApply_)#replace(V, n(A,B)).replace(sigmaApply, sigmaApply_)

lambda(sigma_apply(Lambda((X, W), V)))

In [70]:
# GOAL: want both sigma_apply to be in ---> form composed with the above x,w ---> V form
#lambd(sigmaApply(V)).replace(V, Vs).replace(sigmaApply, sigmaApply_).replace(Vs, V(A,B))### ERROR
lambd(sigmaApply(n(A,B))).replace(n(A,B), VL)
sigmaApply_(A)
sigmaApply_L(A)

Lambda(_d, sigma(_d)).(X)

In [71]:
sigmaApply(Vs).replace(sigmaApply, sigmaApply_)

Lambda(_d, sigma(_d)).(Vs)

In [72]:
sigmaApply(VL(A,B)).replace(sigmaApply, sigmaApply_)#.replace(V(A,B), V)#.subs({sigmaApply: sigmaApply_L})

Lambda(_d, sigma(_d)).(V)

In [73]:
#sigmaApply(Vs).subs({Vs : V, sigmaApply: sigmaApply_L}) ### ERROR must be matrix instance
#sigmaApply(Vs).replace(sigmaApply , sigmaApply_L).subs({Vs : V})
#sigmaApply(V).replace(sigmaApply, sigmaApply_L)

In [74]:

sa = Lambda((A,B), VL)
sa

Lambda((X, W), Lambda((X, W), V))

In [75]:
### ALTERNATE try of declaring a sigma-apply kind of function
#sas = Lambda((A,B), Vs.applyfunc(sigma))

In [76]:
Lambda((A,B), sigma(VL))

Lambda((X, W), sigma(Lambda((X, W), V)))

In [77]:
Lambda((A,B), sigma(VL)).diff(A) # nothing useful with this format, and weird-wrong since doesn't do chain rule wi.r. to sigma

Derivative(Lambda((X, W), sigma(Lambda((X, W), V))), X)

In [78]:
Lambda((A,B), sigma(VL(A,B)))

Lambda((X, W), sigma(V))

In [79]:
sas = Lambda((A,B), VL(A,B).applyfunc(sigma))

sas

Lambda((X, W), Lambda(_d, sigma(_d)).(V))

In [80]:
# YAY this works now I can replace MATRIX SYMBOLS with ordinary sympy LAMBDAS (replace cano only replace same kind of thing / type)
sigma(Vs).subs(Vs, VL)
#

sigma(Lambda((X, W), V))

In [81]:
sas(A,B)

Lambda(_d, sigma(_d)).(V)

In [82]:
# A.applyfunc(sigma).subs(A, VL)# subs method doesn't work here with applyfunc
L

lambda(sigma_apply(v(X, W)))

In [83]:
#sas(A,B).replace(V, V(A,B))

In [84]:
sigmaApply_L

Lambda(M, Lambda(_d, sigma(_d)).(M))

In [85]:
sigmaApply_L(M)

Lambda(_d, sigma(_d)).(M)

In [86]:
#sigmaApply_LFake = Lambda(M, M.applyfunc(sigma))
sigmaApply(M).replace(sigmaApply, sigmaApply_L)

Lambda(_d, sigma(_d)).(M)

In [87]:
#sigmaApply(M).replace(sigmaApply, sigmaApply_).subs(M, n(A,B))
n = Function("v", applyfunc=True)
#sigmaApply_(Vs.subs(Vs, Lambda((A,B), n(A,B))))
from sympy import lambdify
#sigma(lambdify([A,B], n(A,B)))

#inner = Lambda((A,B), n(A,B)); inner

#sigmaApply_(n(A,B))
#sigmaApply(inner).replace(sigmaApply, Lambda(A, sigma(A)))

In [88]:
#sigmaApply_L(M).subs(M, inner)
Lambda(d, sigma(d))

Lambda(d, sigma(d))

In [89]:
### CLOSEST ever gotten to function composition (?) with sympy ....
#Lambda(d, sigma(inner))

In [90]:
#Lambda(d, sigma(inner)).diff(A)

In [91]:
#Lambda(d, sigma(inner)).replace(inner, vL(A,B)).diff(A)

In [92]:
# sigmaApply_L(M).subs(M, VL)# new subs method fails here too
#sigmaApply_(M).subs(M, VL)

In [93]:
sigmaApply_L(M).diff(M)

Lambda(_d, Derivative(sigma(_d), _d)).(M)

In [94]:
sigma(VL)#.replace(V, V(A,B))

sigma(Lambda((X, W), V))

In [95]:
sigma(VL).replace(VL, VL(A,B))

sigma(V)

In [96]:
#sigma(V).replace(V, VL)

In [97]:
f = Function('f')
xtoxL = Lambda(a, a)
xtox = lambda a: a

f(x).subs({x : xtoxL})

f(Lambda(_x, _x))

In [98]:
f(x).subs(x, xtox)# works but below one with replace doesn't. When replacing arg with function uses SUBS without dictionary (instead of replace)

f(x)

In [99]:
# f(x).replace(x, xtox)### ERROR xtox expects one positional argument ( I think replace only replaces the same kind of thing, never for instance a matrix symbol for a function or vice versa. the replacement needs to be of the same type / kind. But Lambda seems to work (as above))
f(x).replace(x, xtoxL)

f(Lambda(_x, _x))

In [100]:
#lambd(sigmaApply(n(A,B))).replace(n(A,B), Vs).replace(sigmaApply, sigmaApply_).replace(Vs, V)# ### ERROR rec replace must be matrix instance ....

In [101]:
### METHOD 0: the matrix diff rule in the most abstract form possible
n = Function("v", applyfunc=True) # necessary
L = lambd(sigmaApply(n(A,B)))

lambd_L = Lambda(A, sum(A))

lambd_L(A)

X[0, 0] + X[0, 1] + X[0, 2] + X[1, 0] + X[1, 1] + X[1, 2] + X[2, 0] + X[2, 1] + X[2, 2]

In [102]:
lambd_L(sigmaApply(n(A,B)))#.replace(n, vL).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_L)

(sigma_apply(v(X, W)))[0, 0] + (sigma_apply(v(X, W)))[0, 1] + (sigma_apply(v(X, W)))[0, 2] + (sigma_apply(v(X, W)))[1, 0] + (sigma_apply(v(X, W)))[1, 1] + (sigma_apply(v(X, W)))[1, 2] + (sigma_apply(v(X, W)))[2, 0] + (sigma_apply(v(X, W)))[2, 1] + (sigma_apply(v(X, W)))[2, 2]

In [103]:
L.replace(n,vL).replace(sigmaApply, sigmaApply_).diff(A)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)*W.T

In [104]:
### SUCCESS! We see now that the matrix chain rule indeed makes the X transpose factor out on the left!!! (while compared to the above, the matrix transpose W^T factors out on the right, just like the book says (page 45 in the NOTE section of Seth Weidman book))
L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [105]:
# Not showing ???
L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B).replace(lambd, lambd_L)

Subs(Derivative(_xi_1[0, 0] + _xi_1[0, 1] + _xi_1[0, 2] + _xi_1[1, 0] + _xi_1[1, 1] + _xi_1[1, 2] + _xi_1[2, 0] + _xi_1[2, 1] + _xi_1[2, 2], _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [106]:
#L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B).replace(B,W).replace(A,X) # ## ERROR non commutative scalars in matrix
# L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B).replace(lambd, lambd_).replace(B,W).replace(A,X)# ## ERROR dummy object not iterable

In [107]:

#L.replace(n,vL).replace(sigmaApply, sigmaApply_).diff(A).replace(lambd, lambd_) ### ERROR: dummy object not iterable (probably means when in the above expression we have epsilon = sigmaApply(XW) that we cannot iterate over this expression)

In [108]:
# Replacing lambda first: BAD
#L.replace(n, v).replace(lambd, lambd_) ## ERROR sigma apply object not ieterable
# Replacing sigma first: BAD
# L.replace(sigmaApply, sigmaApply_)### ERROR v object has no attribute applyfunc

In [109]:
# Replacing n first: GOOD (need to go from inner nesting to outermost function, never any other way)
L.replace(n, v).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_)

sigma(W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + sigma(W[0, 0]*X[1, 0] + W[1, 0]*X[1, 1] + W[2, 0]*X[1, 2]) + sigma(W[0, 0]*X[2, 0] + W[1, 0]*X[2, 1] + W[2, 0]*X[2, 2]) + sigma(W[0, 1]*X[0, 0] + W[1, 1]*X[0, 1] + W[2, 1]*X[0, 2]) + sigma(W[0, 1]*X[1, 0] + W[1, 1]*X[1, 1] + W[2, 1]*X[1, 2]) + sigma(W[0, 1]*X[2, 0] + W[1, 1]*X[2, 1] + W[2, 1]*X[2, 2])

In [110]:
# ### END RESULT of METHOD 2:
L.replace(n, v).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).diff(Matrix(A))

Matrix([
[W[0, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + W[0, 1]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 1]*X[0, 0] + W[1, 1]*X[0, 1] + W[2, 1]*X[0, 2]), W[1, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + W[1, 1]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 1]*X[0, 0] + W[1, 1]*X[0, 1] + W[2, 1]*X[0, 2]), W[2, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + W[2, 1]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 1]*X[0, 0] + W[1, 1]*X[0, 1] + W[2, 1]*X[0, 2])],
[W[0, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[1, 0] + W[1, 0]*X[1, 1] + W[2, 0]*X[1, 2]) + W[0, 1]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 1]*X[1, 0] + W[1, 1]*X[1, 1] + W[2, 1]*X[1, 2]), W[1, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[1, 0] + W[1, 0]*X[1, 1] + W[2, 0]*X[1, 2]) + W[1, 1]*Subs(Derivat

Compare the above matrix symbol way with the Lsum way:

### END RESULT of METHOD 1:

In [111]:
#Lsum = L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_)

L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD).diff(X)#.subs(specToElemD)

Matrix([
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_11 + w_22*x_12 + w_32*x_13), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_11 + w_22*x_12 + w_32*x_13), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13) + w_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_11 + w_22*x_12 + w_32*x_13)],
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_21 + w_21*x_22 + w_31*x_23) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_21 + w_22*x_22 + w_32*x_23), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_21 + w_21*x_22 + w_31*x_23) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_12*x_21 + w_22*x_22 + w_32*x_23), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, w_11*x_21 + w_21*x_22 + w_31*x_23) + w_32*Subs(Derivative(sigma(_xi_1), _

COMPARING METHOD 0 (abstract way) with METHOD 2 (direct way) when differentiating .w.r.t to X vs. w.r.t to W
### With respect to X (abstract)

In [112]:
L.replace(n,vL).replace(sigmaApply, sigmaApply_).diff(A)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)*W.T

### With respect to X (direct)

In [113]:
L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD).diff(X).subs(specToElemD)

Matrix([
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + w_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12)],
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + w_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22)],
[w_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31) + w_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32), w_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31) + w_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32), w_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31) + w_32*Sub

### With respect to W (abstract)

In [114]:
L.replace(n,vL).replace(sigmaApply, sigmaApply_).diff(B)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

### With respect to W (direct)

In [115]:
L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD).diff(W).subs(specToElemD)

Matrix([
[x_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32)],
[x_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32)],
[x_13*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_23*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_33*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_13*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_23*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_33*

### NEXT: try to substitute the X, W matrices step by step to see if you can come to the same result as the direct forms above (from method 2 or 1)

In [116]:
from sympy import simplify, expand

#simplify(L.replace(n,vL).replace(sigmaApply, sigmaApply_).diff(B).subs({A:X, B:W})) ### ERROR max recursion depth exceeded
L.replace(n,vL).replace(sigmaApply, sigmaApply_).diff(B).subs({A:X, B:W})

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]])*Matrix([
[w_11, w_12],
[w_21, w_22],
[w_31, w_32]])))*Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]).T*Lambda(_d, Derivative(sigma(_d), _d)).(Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]])*Matrix([
[w_11, w_12],
[w_21, w_22],
[w_31, w_32]]))

In [117]:
L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B)#.subs({A:X, B:W})

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [118]:
L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [119]:
#L.replace(n,v).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_)

In [120]:
#L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B).subs({A:X, B:W}).replace(lambd, lambd_) ### ERROR dummy object not iterable
L.replace(n,v).diff(A)

Subs(Derivative(sigma_apply(_xi_1), _xi_1), _xi_1, X*W)*Derivative(lambda(sigma_apply(X*W)), sigma_apply(X*W))*Derivative(X*W, X)

In [121]:
#L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(A).replace(A,Matrix(A))##ERROR noncommutative matrix scalars
# WANT: to be able to do diff and have the expression come out as above with X^T on left and W^T on right, when using just this form, with abstract form v:
L.replace(A,A.T).replace(B,B.T)

lambda(sigma_apply(v(X.T, W.T)))

In [122]:
# Error if applying sigma to the v function because it sais v has no attribute applyfunc to trying now to making it have the attriute applyfunc.
y = Function('y', applyfunc=True, real=True)



In [123]:
Ly = lambd(sigmaApply(y(A,B)))
Ly

lambda(sigma_apply(y(X, W)))

In [124]:

Ly.replace(A,A.T).replace(B,B.T)#.replace(sigmaApply, sigmaApply_)

lambda(sigma_apply(y(X.T, W.T)))

In [125]:
# TODO next step: to apply the sigma to get that applied functor expression but here get error saying bol object not callable ...??
Ly.replace(A,A.T).replace(B,B.T)#.replace(sigmaApply, sigmaApply_)

lambda(sigma_apply(y(X.T, W.T)))

In [126]:
# TODO always get fatal python error here, as if it can't deal with two matrix args!!
#Ly.replace(A,A.T).replace(B,B.T).diff(A)

#siga2 = Lambda(a, siga(a))

In [127]:
Ly.replace(A, A.T).replace(B, b).diff(b)#.replace(sigmaApply, siga)

Derivative(lambda(sigma_apply(y(X.T, b))), sigma_apply(y(X.T, b)))*Derivative(sigma_apply(y(X.T, b)), y(X.T, b))*(Derivative(y(X.T, b), b) + 0)

In [128]:
L.replace(n, vN).replace(sigmaApply, sigmaApply_).subs(elemToMatArgD)

lambda(Matrix([
[sigma(n_11(X, W)), sigma(n_12(X, W))],
[sigma(n_21(X, W)), sigma(n_22(X, W))],
[sigma(n_31(X, W)), sigma(n_32(X, W))]]))

In [129]:
#L.replace(n, vN).replace(sigmaApply, sigmaApply_).subs(elemToMatArgD).diff(A)## ERROR: max recursion depth eceeded

L.replace(n, vN).replace(sigmaApply, sigmaApply_).subs(elemToMatArgD).diff(Matrix(3,2,list(elemToMatArgD.values())))

Matrix([
[Matrix([
[Derivative(sigma(n_11(X, W)), n_11(X, W))*Derivative(lambda(Matrix([
[sigma(n_11(X, W)), sigma(n_12(X, W))],
[sigma(n_21(X, W)), sigma(n_22(X, W))],
[sigma(n_31(X, W)), sigma(n_32(X, W))]])), Matrix([
[sigma(n_11(X, W)), sigma(n_12(X, W))],
[sigma(n_21(X, W)), sigma(n_22(X, W))],
[sigma(n_31(X, W)), sigma(n_32(X, W))]])), 0],
[                                                                                                                                                                                                                                                                                                                                 0, 0],
[                                                                                                                                                                                                                                                                                                                                 0

In [130]:
A.applyfunc(sigma)

Lambda(_d, sigma(_d)).(X)

In [131]:
sigma = Function("sigma", applyfunc=True, bool=False)

In [132]:
sigma.__dict__

mappingproxy({'applyfunc': True,
              'bool': False,
              '_kwargs': {'applyfunc': True, 'bool': False},
              '__module__': None,
              '__doc__': None,
              'name': 'sigma',
              '_sage_': <sympy.core.function.UndefSageHelper at 0x7fea7613ae50>,
              '_nargs': None,
              '__sympy__': <property at 0x7fea74d761d0>,
              '_explicit_class_assumptions': {},
              'default_assumptions': {},
              '_prop_handler': {'extended_positive': <function sympy.core.expr.Expr._eval_is_extended_positive(self)>,
               'positive': <function sympy.core.expr.Expr._eval_is_positive(self)>,
               'negative': <function sympy.core.expr.Expr._eval_is_negative(self)>,
               'extended_negative': <function sympy.core.expr.Expr._eval_is_extended_negative(self)>,
               'commutative': <function sympy.core.function.Function._eval_is_commutative(self)>}})

In [133]:
Ly = lambd(sigmaApply(y(A,B))); Ly

lambda(sigma_apply(y(X, W)))

In [134]:
(X*W).applyfunc(sigma)

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [135]:
(A*B).applyfunc(sigma)

Lambda(_d, sigma(_d)).(X*W)

In [136]:
siga(A)
#A.applyfunc(siga) ### ERROR dumy object has no attribute shape

Matrix([
[sigma(X[0, 0]), sigma(X[0, 1]), sigma(X[0, 2])],
[sigma(X[1, 0]), sigma(X[1, 1]), sigma(X[1, 2])],
[sigma(X[2, 0]), sigma(X[2, 1]), sigma(X[2, 2])]])

In [137]:
y = Function("y", applyfunc = True, bool=False, shape=(3,3))
y.shape

(3, 3)

In [138]:
# siga(y(A,B))### ERROR: function y is not subscriptable

In [139]:
Ly.subs({A:a,B:b}).diff(b).subs({a:A, b:B})#.replace(sigmaApply, sigmaApply_)

Derivative(lambda(sigma_apply(y(X, W))), sigma_apply(y(X, W)))*Derivative(sigma_apply(y(X, W)), y(X, W))*Derivative(y(X, W), W)

In [140]:
L.replace(A,a).replace(B,b).diff(b).subs({a:A,b:B})#.replace(sigmaApply, sigmaApply_)#.diff(b)

Derivative(lambda(sigma_apply(v(X, W))), sigma_apply(v(X, W)))*Derivative(sigma_apply(v(X, W)), v(X, W))*Derivative(v(X, W), W)

In [141]:
sigma = Function("sigma", applyfunc=True, real=True)
sigmaApply_ = lambda mat: mat.applyfunc(sigma)
L = lambd(sigmaApply(n(A,B)))

#L.replace(A,a).replace(B,b).diff(b).subs({a:A,b:B}).replace(sigmaApply, sigmaApply_)
L.replace(n, v).replace(sigmaApply, sigmaApply_).diff(A)
#m = Symbol("m", shape=(3,2))
#m.shape

#sigmaApply_3 = Lambda(m, siga(m))

#L.replace(A,a).replace(B,b).diff(b).replace(b,B).replace(a,A).subs({n:vL}).replace(sigmaApply, sigmaApply_2) ### ERROR: Dummy object has no attribute shape

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)*W.T

In [142]:
# Ly.replace(B, b).diff(A)#.replace(sigmaApply, siga)### ERROR noncommutative matrix scalars not supported
Ly.replace(A, A.T).replace(B, b).diff(b).replace(b, B).replace(A.T, A)#.replace(sigmaApply, siga)

Derivative(lambda(sigma_apply(y(X, W))), sigma_apply(y(X, W)))*Derivative(sigma_apply(y(X, W)), y(X, W))*(Derivative(y(X, W), W) + 0)

In [143]:
#Ly.replace(B,b).diff(b).replace(b,B) ### ERROR

In [144]:
# NEXT: try to replace the sigma apply, not working
n.__dict__

mappingproxy({'applyfunc': True,
              '_kwargs': {'applyfunc': True},
              '__module__': None,
              '__doc__': None,
              'name': 'v',
              '_sage_': <sympy.core.function.UndefSageHelper at 0x7fea7613ae50>,
              '_nargs': None,
              '__sympy__': <property at 0x7fea74bf9170>,
              '_explicit_class_assumptions': {},
              'default_assumptions': {},
              '_prop_handler': {'extended_positive': <function sympy.core.expr.Expr._eval_is_extended_positive(self)>,
               'positive': <function sympy.core.expr.Expr._eval_is_positive(self)>,
               'negative': <function sympy.core.expr.Expr._eval_is_negative(self)>,
               'extended_negative': <function sympy.core.expr.Expr._eval_is_extended_negative(self)>,
               'commutative': <function sympy.core.function.Function._eval_is_commutative(self)>}})

In [145]:
y.__dict__
# TODO HERE
#https://stackoverflow.com/questions/12614334/typeerror-bool-object-is-not-callable

mappingproxy({'applyfunc': True,
              'bool': False,
              'shape': (3, 3),
              '_kwargs': {'applyfunc': True, 'bool': False, 'shape': (3, 3)},
              '__module__': None,
              '__doc__': None,
              'name': 'y',
              '_sage_': <sympy.core.function.UndefSageHelper at 0x7fea7613ae50>,
              '_nargs': None,
              '__sympy__': <property at 0x7fea74a22110>,
              '_explicit_class_assumptions': {},
              'default_assumptions': {},
              '_prop_handler': {'extended_positive': <function sympy.core.expr.Expr._eval_is_extended_positive(self)>,
               'positive': <function sympy.core.expr.Expr._eval_is_positive(self)>,
               'negative': <function sympy.core.expr.Expr._eval_is_negative(self)>,
               'extended_negative': <function sympy.core.expr.Expr._eval_is_extended_negative(self)>,
               'commutative': <function sympy.core.function.Function._eval_is_commutative(

In [146]:
from sympy import diff
# ### WARNING: this only works when size(X) == size(Y) else since size(W) != size(X) cannot subst B with W, so this operation won't work in my case.

#X = Matrix(3,3, lambda i,j: Symbol("x_{}{}".format(i+1,j+1))); Matrix(X)
# Create another matrix instead of W so that it matches size of X during diff(X) operation, since otherwise the diff by X doesn't work, says X and W need to be same size.

Wtemp = Matrix(*X.shape, lambda i,j: Symbol("t_{}{}".format(i+1,j+1))); Matrix(Wtemp)

Matrix([
[t_11, t_12, t_13],
[t_21, t_22, t_23],
[t_31, t_32, t_33]])

In [147]:
#L.subs({A:X, B:Wtemp}).diff(X)[0,0][0,0].replace(n,vN).replace(sigmaApply, sigmaApply_)#.doit()
#diff(L.replace(A,A.T), A) # ERROR max recursion depth exceeded

In [148]:
#Lmat = L.subs({A:X, B:Wtemp}).diff(X).subs({X:A, Wtemp: B}); Lmat #replace(X, A).replace(Y,B); Lmat
# NOTE need to do replace at the end (instead of subs) else it says unhasable type mutabledensematrix.
Lmat = L.subs({A:X, B:Wtemp}).diff(X).replace(X, A).replace(Wtemp,B); Lmat
#L.diff(A) # HELL ON THE EDITOR NEVER TRY THIS AGAIN

Matrix([
[Matrix([
[Derivative(v(X, W), X)*Derivative(sigma_apply(v(X, W)), v(X, W))*Derivative(lambda(sigma_apply(v(X, W))), sigma_apply(v(X, W))), 0, 0],
[                                                                                                                              0, 0, 0],
[                                                                                                                              0, 0, 0]]), Matrix([
[0, Derivative(v(X, W), X)*Derivative(sigma_apply(v(X, W)), v(X, W))*Derivative(lambda(sigma_apply(v(X, W))), sigma_apply(v(X, W))), 0],
[0,                                                                                                                               0, 0],
[0,                                                                                                                               0, 0]]), Matrix([
[0, 0, Derivative(v(X, W), X)*Derivative(sigma_apply(v(X, W)), v(X, W))*Derivative(lambda(sigma_apply(v(X, W))), sigma_apply(v(X, W)))],


In [149]:
#L.replace(A,X).replace(B,W)

In [150]:
# Method 2 approach for comparison:
#L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD).diff(X)#.subs(specToElemD)

In [151]:
elem = Lmat[0,0][0,0];elem

Derivative(v(X, W), X)*Derivative(sigma_apply(v(X, W)), v(X, W))*Derivative(lambda(sigma_apply(v(X, W))), sigma_apply(v(X, W)))

In [152]:
#Lmat.replace(n, vL) # error can't calc deriv .w.r.t to x11*w11 +...
# Lmat.replace(n, v) # error can't calc deriv .w.r.t to x11*w11 +...
elem.subs(n, vL)

Subs(Derivative(sigma_apply(_xi_0), _xi_0), _xi_0, X*W)*Derivative(X*W, X)*Derivative(lambda(sigma_apply(X*W)), sigma_apply(X*W))

In [153]:
#elem.replace(n, v) # error cannot deriv wrt to X*W

In [154]:
Selem

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [155]:
# use replace n with vN instead of subs n with vL to get less specific output so it is easier to see since vL returns the xww*w11 +.... expressions
elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_)

Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]))*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))*Derivative(lambda(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])), Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))

In [156]:
# Making matrix symbols again
Ss = MatrixSymbol('S', 3,2) #n by p
Ns = MatrixSymbol('N', 3,2) #n by p



short = elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_).replace(X,A).replace(Nelem, Ns).replace(Selem ,Ss)
short

Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)*Derivative(lambda(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])), Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))

In [157]:
# Now going back to matrix form just to apply the last function LAMBDA
elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_)

Derivative(sigma(n_11) + sigma(n_12) + sigma(n_21) + sigma(n_22) + sigma(n_31) + sigma(n_32), Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))*Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]))*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))

In [158]:
# Making each of the n_ijs a function
#elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD)

Matrix(elemToSpecFuncArgs)

Matrix([
[n_11, n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_12, n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_21, n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_22, n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_31, n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_32, n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)]])

In [159]:

long = elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecFuncArgsD)
long

Derivative(sigma(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)) + sigma(n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)) + sigma(n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)) + sigma(n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)) + sigma(n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)) + sigma(n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)), Matrix([
[sigma(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)), sigma(n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32))],
[sigma(n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)), sigma(n_22(x_11, x_12, x_13, x_21, x_22, x_23

In [160]:
# short version again:
short

Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)*Derivative(lambda(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])), Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))

In [161]:
# long.doit() # error as base exp thing

In [162]:
# Trying step by step replacement approach:
elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).replace(Nelem, Ns).replace(X,A)

Derivative(sigma(n_11) + sigma(n_12) + sigma(n_21) + sigma(n_22) + sigma(n_31) + sigma(n_32), Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))*Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)

In [163]:
# Seeing if replacing the order of replacing Ns matrix with Xs matrix makes a difference: ...
step = elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).replace(Nelem, Ns).replace(X,A).doit()
step

Matrix([
[Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N), Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)],
[Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N), Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)],
[Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N), Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)]])

In [164]:
elem.subs({A:X, B:W}).replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).replace(X,A).replace(Nelem, Ns).doit()

Matrix([
[Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N), Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)],
[Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N), Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)],
[Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N), Derivative(N, X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), N)]])

In [165]:
step.replace(Ns, Nelem)

Matrix([
[Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])), Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))],
[Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])), Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))],
[Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), X)*Derivative(

In [166]:
#step.replace(Ns, Nelem).replace(A,X).doit()#error immutable dense array has no attribute as base exp ...
elem2 = step[0,0].replace(Ns, Nelem)
elem2.replace(A,X).subs(elemToSpecFuncArgsD)

Derivative(Matrix([
[n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)]]), Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]))*Derivative(Matrix([
[sigma(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)), sigma(n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32))],
[sigma(n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w

In [167]:
#elem2.replace(A,X).subs(elemToSpecFuncArgsD).doit()
F = Nelem.subs(elemToSpecFuncArgsD); F

Matrix([
[n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)]])

In [168]:
F[0,0].diff(X[0,0])

Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11)

In [169]:
F[0,0].diff(X)

Matrix([
[Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11), Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_12), Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_13)],
[Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_21), Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_22), Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_23)],
[Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_31), Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_32), Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11

In [170]:
F.diff(X)

[[[[Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11), Derivative(n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11)], [Derivative(n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11), Derivative(n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11)], [Derivative(n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11), Derivative(n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_11)]], [[Derivative(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_12), Derivative(n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), x_12)], [Derivative(n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11

In [171]:
argsToSpecD = dict(zip(elemToSpecFuncArgsD.values(), elemToSpecD.values()))
argsToSpec = list(argsToSpecD.items())
Matrix(argsToSpec)

Matrix([
[n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), w_11*x_11 + w_21*x_12 + w_31*x_13],
[n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), w_12*x_11 + w_22*x_12 + w_32*x_13],
[n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), w_11*x_21 + w_21*x_22 + w_31*x_23],
[n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), w_12*x_21 + w_22*x_22 + w_32*x_23],
[n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), w_11*x_31 + w_21*x_32 + w_31*x_33],
[n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), w_12*x_31 + w_22*x_32 + w_32*x_33]])

In [172]:
F[0,0].diff(X[0,0]).subs(argsToSpecD)#.subs({elemToSpecFuncArgs[0][1] : Nspec[0,0]})

Derivative(w_11*x_11 + w_21*x_12 + w_31*x_13, x_11)

In [173]:
F[0,0].diff(X[0,0]).subs(argsToSpecD).doit()

w_11

In [174]:
# NOTE: using diff did not work, said immutable dense array cannot be subs-ed
derive_by_array(F, X).subs(argsToSpecD)

[[[[Derivative(w_11*x_11 + w_21*x_12 + w_31*x_13, x_11), Derivative(w_12*x_11 + w_22*x_12 + w_32*x_13, x_11)], [Derivative(w_11*x_21 + w_21*x_22 + w_31*x_23, x_11), Derivative(w_12*x_21 + w_22*x_22 + w_32*x_23, x_11)], [Derivative(w_11*x_31 + w_21*x_32 + w_31*x_33, x_11), Derivative(w_12*x_31 + w_22*x_32 + w_32*x_33, x_11)]], [[Derivative(w_11*x_11 + w_21*x_12 + w_31*x_13, x_12), Derivative(w_12*x_11 + w_22*x_12 + w_32*x_13, x_12)], [Derivative(w_11*x_21 + w_21*x_22 + w_31*x_23, x_12), Derivative(w_12*x_21 + w_22*x_22 + w_32*x_23, x_12)], [Derivative(w_11*x_31 + w_21*x_32 + w_31*x_33, x_12), Derivative(w_12*x_31 + w_22*x_32 + w_32*x_33, x_12)]], [[Derivative(w_11*x_11 + w_21*x_12 + w_31*x_13, x_13), Derivative(w_12*x_11 + w_22*x_12 + w_32*x_13, x_13)], [Derivative(w_11*x_21 + w_21*x_22 + w_31*x_23, x_13), Derivative(w_12*x_21 + w_22*x_22 + w_32*x_23, x_13)], [Derivative(w_11*x_31 + w_21*x_32 + w_31*x_33, x_13), Derivative(w_12*x_31 + w_22*x_32 + w_32*x_33, x_13)]]], [[[Derivative(w_11*

In [175]:
derive_by_array(F, X).subs(argsToSpecD).doit()

[[[[w_11, w_12], [0, 0], [0, 0]], [[w_21, w_22], [0, 0], [0, 0]], [[w_31, w_32], [0, 0], [0, 0]]], [[[0, 0], [w_11, w_12], [0, 0]], [[0, 0], [w_21, w_22], [0, 0]], [[0, 0], [w_31, w_32], [0, 0]]], [[[0, 0], [0, 0], [w_11, w_12]], [[0, 0], [0, 0], [w_21, w_22]], [[0, 0], [0, 0], [w_31, w_32]]]]

In [176]:
derive_by_array(F, W).subs(argsToSpecD).doit()




[[[[x_11, 0], [x_21, 0], [x_31, 0]], [[0, x_11], [0, x_21], [0, x_31]]], [[[x_12, 0], [x_22, 0], [x_32, 0]], [[0, x_12], [0, x_22], [0, x_32]]], [[[x_13, 0], [x_23, 0], [x_33, 0]], [[0, x_13], [0, x_23], [0, x_33]]]]

In [177]:
elem2

Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))

In [178]:

funcMat = elem2.subs(elemToSpecFuncArgsD).replace(A,X)#.diff(X)
funcMat

Derivative(Matrix([
[n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32), n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)]]), Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]))*Derivative(Matrix([
[sigma(n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)), sigma(n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32))],
[sigma(n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w

In [179]:
#funcMat.doit() # error
#derive_by_array(funcMat, X)

In [180]:
funcMat = elem2.subs(elemToSpecFuncD).replace(A,X)#.diff(X)
funcMat

Derivative(Matrix([
[n_11(w_11*x_11 + w_21*x_12 + w_31*x_13), n_12(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[n_21(w_11*x_21 + w_21*x_22 + w_31*x_23), n_22(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[n_31(w_11*x_31 + w_21*x_32 + w_31*x_33), n_32(w_12*x_31 + w_22*x_32 + w_32*x_33)]]), Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]))*Derivative(Matrix([
[sigma(n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)), sigma(n_12(w_12*x_11 + w_22*x_12 + w_32*x_13))],
[sigma(n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)), sigma(n_22(w_12*x_21 + w_22*x_22 + w_32*x_23))],
[sigma(n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)), sigma(n_32(w_12*x_31 + w_22*x_32 + w_32*x_33))]]), Matrix([
[n_11(w_11*x_11 + w_21*x_12 + w_31*x_13), n_12(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[n_21(w_11*x_21 + w_21*x_22 + w_31*x_23), n_22(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[n_31(w_11*x_31 + w_21*x_32 + w_31*x_33), n_32(w_12*x_31 + w_22*x_32 + w_32*x_33)]]))

In [181]:
#funcMat.doit() # same error
#elem2.subs(elemToSpecFuncD).doit() # error
elem2

Derivative(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]), X)*Derivative(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]), Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))

In [182]:
# elem2.replace(A,X).doit() # error

In [183]:
#elem2.replace(A,a).doit()#.subs(elemToSpecFuncArgsD).doit()
# ERROR everywhere what next todo? this approach worked before, where I make w.r.t. thing a real matrix, and leave the others a symbol so why isn't it working now?

In [184]:
#elem2.replace(A,X).subs(elemToSpecFuncD).doit()
# ERROR this has to work though! Then can simply replace n_ijs with lambda



In [185]:
#elem2.subs(elemToMatArgD).doit()#ERROR max recursion depth exceeeded